In [1]:
# !pip install umap
# !pip install umap-learn
# !pip install nb_black
# !pip install plotly

In [2]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from umap import UMAP

from prince import MCA, FAMD

import plotly.express as px
from ipywidgets import interact

import seaborn as sns
import matplotlib.pyplot as plt

import ast

%matplotlib inline

<IPython.core.display.Javascript object>

In [ ]:
spotify = pd.read_excel("data/Hot_100_Audio_Features.xlsx")
spotify.head()

In [ ]:
spotify = spotify.drop_duplicates()

In [ ]:
spotify = spotify.drop(
    columns=[
        "spotify_track_id",
        "spotify_track_preview_url",
    ]
)

In [ ]:
spotify.shape

In [ ]:
spotify.head()

In [ ]:
spotify.dtypes

In [ ]:
spotify.isna().mean()

# Fill null values

In [ ]:
spotify["spotify_track_album"].fillna(" ", inplace=True)
spotify["spotify_genre"].fillna("[]", inplace=True)

# Reformat the genre list
spotify["spotify_genre_list"] = spotify["spotify_genre"].apply(ast.literal_eval)

In [ ]:
# Try to change all columns to numeric
# if theres an error, ignore it and move to next column
spotify = spotify.apply(pd.to_numeric, errors="ignore")

In [ ]:
spotify["spotify_track_explicit"].fillna("Not filled", inplace=True)

# Fill N/As with mean values
null_cols = [
    "spotify_track_duration_ms",
    "spotify_track_popularity",
    "danceability",
    "energy",
    "key",
    "loudness",
    "mode",
    "speechiness",
    "acousticness",
    "instrumentalness",
    "liveness",
    "valence",
    "tempo",
    "time_signature",
]

for col in null_cols:
    spotify[col].fillna(spotify[col].mean(), inplace=True)

In [ ]:
spotify

In [ ]:
# Isolate numbers
nums = spotify.select_dtypes("number")

@interact
def scatterplot(
    x=nums.columns, 
    y=nums.columns, 
    hue=spotify['spotify_genre_list'],
    hover_data=['Performer', 'spotify_genre_list']
):
    px.scatter(data_frame=spotify, x=x, y=y, color=hue, hover_data=hover_data)
    plt.show()